# 02 - Tutorial - Requesting match information and converting data.

In the last tutorial, we learned:
- How to do a request on Riot API
- How to use urllib.request module to make a request on a site
- How to use JSON, dump and loads module to serialize and unserialize data, respectively.
- The format that url will be to make an request
- The format that the response will be

Now, we're gonna try to use another API to get useful information.

# Step 1 : Checking out the format of the output of the next API

The next API that we're gonna study is: 

/lol/match/v4/matchlists/by-account/{encryptedAccountId} 

On the MATCH-V4 tab.(https://developer.riotgames.com/apis#match-v4/GET_getMatchlist)

The only required parameter is encryptedAccountID, but, we wanna take a look on the output format.
If you read the IMPLEMENTATION NOTES, you're gonna find out, that this request, returns more than 1 game, as default.
The parameters that indicate how many games you're gonna get are:

beginIndex = (default = 0)
endIndex = (maximun = beginIndex+100)(default = beginIndex + 100)

I'm not going to talk about the other parameters for now. It's not relevant for the next step.

So, we're gonna reuse the code from tutorial 01 to get the data, making some modifications to the new API.

In [1]:
import json # Import the library that handles .json format

# Getting information for the request
with open('01-summonerinfo.txt', 'r') as d:
    summonerInfo = json.loads(d.read())
print(json.dumps(summonerInfo, indent = 7))

{
       "id": "9rBjBXxhkQk4WgZy9IdYOS3rPMvFMghThRWADa-CoXPf",
       "accountId": "PJOkubvbPWpniJ9p02SF1meovFvLLluof1R85shF5fc",
       "puuid": "sXIzTHBtHhvB7F5Tm9nCTHGT46VtsbFOT8Olo5sFtb_8mbFi4B1ytBYLzJuDH9sgg4JC1UBiAUc6BA",
       "name": "Chinen",
       "profileIconId": 523,
       "revisionDate": 1572382204000,
       "summonerLevel": 121
}


If you remember from the last tutorial, the encryptedAccountId needed for this new API is the value of "accountId". So we're gonna use it to get the next request

In [2]:
import urllib.request # Importing the module request
import json # Import the library that handles .json format

# Getting information for the request
region = "br1"
key = "You-Code-Here"
#beginIndex is default 0, so we only need to specify the endIndex
endIndex = "1"

# Creating the url to request(GET)
request = "https://"+region+".api.riotgames.com/lol/match/v4/matchlists/by-account/"+summonerInfo['accountId']+"?api_key="+key+'&endIndex='+endIndex

with urllib.request.urlopen(request) as response:
    data = json.loads(response.read(300).decode('utf-8'))

In [3]:
json.dumps(data, indent = 1)
print(json.dumps(data, indent = 1))

{
 "matches": [
  {
   "platformId": "BR1",
   "gameId": 1785767421,
   "champion": 31,
   "queue": 900,
   "season": 13,
   "timestamp": 1573149951388,
   "role": "DUO_SUPPORT",
   "lane": "NONE"
  }
 ],
 "startIndex": 0,
 "endIndex": 1,
 "totalGames": 149
}


As you can see, most of the information are numbers. Those information can be found in the 2 sites:

https://developer.riotgames.com/docs/lol
https://developer.riotgames.com/apis#match-v4/GET_getMatchlist

# Timestamp
The first atribute that we're gonna convert, is the timestamp.

The timestamp is a long int, and indicates the date in milliseconds, after 1970.

So we're gonna convert this time to a human inteligible date.

The module that we're gonna use is datetime, from python.

A good explanation is found here: https://www.pylenin.com/blogs/mastering-python-datetime/

In [4]:
from datetime import datetime

data["matches"][0]["timestamp"] = datetime.fromtimestamp(data["matches"][0]["timestamp"]//1000) # We're gonna divide the number for 1000 to get the UNIX standard

In [5]:
print(data["matches"][0]["timestamp"])

2019-11-07 15:05:51


As you can see, the format is still not in a good format(BR Time is UTC - 3), so we have to convert it again.

In [6]:
from datetime import datetime,timedelta

timediff = timedelta(hours = 3)

data["matches"][0]["timestamp"] = data["matches"][0]["timestamp"] + timediff

print(data["matches"][0]["timestamp"])

2019-11-07 18:05:51


Now we're going to transform this data into a string, to be serializable

In [7]:
data["matches"][0]["timestamp"] = str(data["matches"][0]["timestamp"])

So now the date is in a good format.

# Champion
The next atribute that we're going to change is the champion. We can find the information about the conversion this integer to a string(name) in the site:
https://developer.riotgames.com/docs/lol#data-dragon_champions

The first thing that we're going to do is to download the information from the site.

In [17]:
import urllib.request

with urllib.request.urlopen("http://ddragon.leagueoflegends.com/cdn/9.21.1/data/en_US/champion.json") as site:
    with open('champion.json', 'w') as file:
        json.dump(json.load(site), file, indent = 1)
        
with open('champion.json', 'r') as file:
    championInfo = json.load(file)

The data downloaded is too big to insert in this notebook. Feel free to open the file and investigate(But I will too).
The information that we are looking for are inside on championInfo['data']['Name of the champion']['key']

In [23]:
print(str(championInfo['data']['Aatrox']['key']))

266


As you can see, the key code for Aatrox is 266.
So we're now, going to change all the number for the names of the champion on the data.

First, we're gonna create a dictionary, that contains the name and the key of the champion.

In [32]:
champ_dict = {}
for i in range(len(championInfo['data'])):
    for champ in championInfo['data']:
        champ_dict[championInfo['data'][champ]['key']] = champ

Now, we have the dictionary to use to convert the keys to champion names.
So we're going to convert

In [33]:
for i in range(len(data['matches'])):
    key = data['matches'][i]['champion']
    data['matches'][i]['champion'] = champ_dict[str(key)]
    
# Storing the data on an archive
with open('converted_data.json', 'w') as writing:
    json.dump(data, writing, indent = 1)

31


Now, we have the converted data.

In [47]:
with open('converted_data.json', 'r') as r:
    pp = json.load(r)
    print(json.dumps(pp, indent = 1))

{
 "matches": [
  {
   "platformId": "BR1",
   "gameId": 1785767421,
   "champion": "Chogath",
   "queue": 900,
   "season": 13,
   "timestamp": "2019-11-07 18:05:51",
   "role": "DUO_SUPPORT",
   "lane": "NONE"
  }
 ],
 "startIndex": 0,
 "endIndex": 1,
 "totalGames": 149
}


As you can see, I played as Chogath on the date: 2019-11-07 18:05:51
This is for now :).

C ya.